In [1]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

from agents.linkedin_lookup_agent import linkedin_url_lookup
from utils.linkedin_info_extract import scrape_linkedin_profile

from tools.tools import get_linkedin_profile_url
from utils.output_parsers import person_intel_parser



from key_storage import key_dict
list(key_dict.keys())

['OPENAI_API_KEY',
 'PINECONE_API_KEY',
 'PINECONE_API_ENV',
 'PROXYCURL_API_KEY',
 'SERPAPI_API_KEY']

In [2]:
info,data = scrape_linkedin_profile("https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/")
print(info)
print()
print(data)

{'public_identifier': 'venetis-pallikaras', 'first_name': 'Venetis', 'last_name': 'Pallikaras', 'occupation': 'Graduate Teaching Assistant at Georgia Institute of Technology', 'headline': 'Data Science | Machine Learning', 'summary': 'no info', 'country_full_name': 'United States of America', 'city': 'Washington DC-Baltimore Area', 'experiences': [{'starts_at': {'day': 1, 'month': 8, 'year': 2022}, 'ends_at': None, 'company': 'Georgia Institute of Technology', 'title': 'Graduate Teaching Assistant', 'description': 'GTA for the course CS 7641-Machine Learning', 'location': 'Atlanta, Georgia, United States'}, {'starts_at': {'day': 1, 'month': 5, 'year': 2022}, 'ends_at': {'day': 31, 'month': 8, 'year': 2022}, 'company': 'GE', 'title': 'Intern', 'description': 'Summer Practicum', 'location': 'Virtual'}, {'starts_at': {'day': 1, 'month': 7, 'year': 2018}, 'ends_at': {'day': 30, 'month': 9, 'year': 2018}, 'company': 'NCSR "DEMOKRITOS"\u200b', 'title': 'Intern', 'description': '•\tIntern in 

In [4]:
info,data = scrape_linkedin_profile("https://www.linkedin.com/in/brokenlinktest/")
info,data 

couldn't retrieve information-1
<PreparedRequest [GET]>
<Response [404]>


(None, None)

In [5]:
res = linkedin_url_lookup(name= "Venetis Pallikaras") 
print("result is:", res)



> Entering new AgentExecutor chain...
I need to use the tool that can crawl Google for a linkedin profile page to find the URL for Venetis Pallikaras.
Action: Crawl Google for linkedin profile page
Action Input: "Venetis Pallikaras linkedin"
Observation: https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/
Thought:I have found the URL for Venetis Pallikaras' Linkedin profile page.
Final Answer: https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/

> Finished chain.
result is: https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/


In [6]:
res = linkedin_url_lookup(name= "Venetis Pallikaras", occupation = "Georgia Tech") 
print("result is:", res)



> Entering new AgentExecutor chain...
I need to use the tool Crawl Google for linkedin profile page to find the URL for Venetis Pallikaras' Linkedin profile page.
Action: Crawl Google for linkedin profile page
Action Input: 'Venetis Pallikaras, Georgia Tech'
Observation: https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/
Thought:I found the URL for Venetis Pallikaras' Linkedin profile page.
Final Answer: https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/

> Finished chain.
result is: https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/


In [7]:
get_linkedin_profile_url("Venetis Pallikaras")

'https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/'

In [3]:
summary_template = """
         given the information about a person from linkedin {information},I want you to create:
         1. a short summary
         2. two interesting facts about them
         \n
     """

llm = ChatOpenAI(
                 temperature=0,
                 model_name = "gpt-3.5-turbo",
                 openai_api_key = key_dict["OPENAI_API_KEY"],
                 )

summary_prompt_template = PromptTemplate(
        input_variables=["information"],
        template=summary_template,

    )

chain = LLMChain(llm=llm, prompt=summary_prompt_template)

In [2]:
chain.run(information=info)

In [22]:
def qna_linkedin_profile(name=None,occupation=None):
    link = linkedin_url_lookup(name= name, occupation = occupation) 
    print("found link is:", link)    
    
    linkedin_info,_ = scrape_linkedin_profile(link)

    #
    summary_template = """
         given the Linkedin information {linkedin_information} about a person, I want you to create:
         1. A short summary about them
         2. Three interesting facts about them
         3. A topic that may interest them
         4. Two creative Ice breakers to open a conversation with them
        \n {format_instructions}
     """
    #format instruction tells the model it needs to follow the pattern thus no need to write it explicitely eg follow this schema

    summary_prompt_template = PromptTemplate(
        input_variables=["linkedin_information"],
        template=summary_template,
        partial_variables={ #information that we provide at the beginning, contrary to input_vars which we fill later
            "format_instructions": person_intel_parser.get_format_instructions()
        },
    )

    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo",
                     openai_api_key = key_dict["OPENAI_API_KEY"],)

    chain = LLMChain(llm=llm, prompt=summary_prompt_template)

    result = chain.run(linkedin_information=linkedin_info)

    #print("scehma explanation:",person_intel_parser.get_format_instructions())
    return result, person_intel_parser.parse(result), linkedin_info, person_intel_parser.parse(result).to_dict()


In [23]:
raw_answer, parsed_answer, raw_linkedin_info, dict_answer = qna_linkedin_profile("Venetis Pallikaras")



> Entering new AgentExecutor chain...
I need to use the tool that can crawl Google for a linkedin profile page to find the URL for Venetis Pallikaras.
Action: Crawl Google for linkedin profile page
Action Input: "Venetis Pallikaras linkedin"
Observation: https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/
Thought:I have found the URL for Venetis Pallikaras' Linkedin profile page.
Final Answer: https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/

> Finished chain.
found link is: https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/


In [24]:
raw_answer

'{\n    "summary": "Venetis Pallikaras is a Graduate Teaching Assistant at Georgia Institute of Technology with a focus on Data Science and Machine Learning. He has experience working as an intern at GE and NCSR DEMOKRITOS in Greece.",\n    "facts": [\n        "Venetis has completed multiple courses and certifications in Data Science, Machine Learning, and Programming.",\n        "He is fluent in both English and Greek.",\n        "Venetis has experience working with big data technologies such as PySpark and Apache Spark."\n    ],\n    "topics_of_interest": [\n        "Data Science",\n        "Machine Learning",\n        "Big Data Technologies"\n    ],\n    "ice_breakers": [\n        "What inspired you to pursue a career in Data Science and Machine Learning?",\n        "I noticed you have experience working with PySpark and Apache Spark. What was your experience like working with these technologies?"\n    ]\n}'

In [25]:
parsed_answer

PersonIntel(summary='Venetis Pallikaras is a Graduate Teaching Assistant at Georgia Institute of Technology with a focus on Data Science and Machine Learning. He has experience working as an intern at GE and NCSR DEMOKRITOS in Greece.', facts=['Venetis has completed multiple courses and certifications in Data Science, Machine Learning, and Programming.', 'He is fluent in both English and Greek.', 'Venetis has experience working with big data technologies such as PySpark and Apache Spark.'], topics_of_interest=['Data Science', 'Machine Learning', 'Big Data Technologies'], ice_breakers=['What inspired you to pursue a career in Data Science and Machine Learning?', 'I noticed you have experience working with PySpark and Apache Spark. What was your experience like working with these technologies?'])

In [26]:
type(parsed_answer)

utils.output_parsers.PersonIntel

In [27]:
parsed_answer.to_dict()

{'summary': 'Venetis Pallikaras is a Graduate Teaching Assistant at Georgia Institute of Technology with a focus on Data Science and Machine Learning. He has experience working as an intern at GE and NCSR DEMOKRITOS in Greece.',
 'facts': ['Venetis has completed multiple courses and certifications in Data Science, Machine Learning, and Programming.',
  'He is fluent in both English and Greek.',
  'Venetis has experience working with big data technologies such as PySpark and Apache Spark.'],
 'topics_of_interest': ['Data Science',
  'Machine Learning',
  'Big Data Technologies'],
 'ice_breakers': ['What inspired you to pursue a career in Data Science and Machine Learning?',
  'I noticed you have experience working with PySpark and Apache Spark. What was your experience like working with these technologies?']}

In [28]:
raw_linkedin_info

{'public_identifier': 'venetis-pallikaras',
 'first_name': 'Venetis',
 'last_name': 'Pallikaras',
 'occupation': 'Graduate Teaching Assistant at Georgia Institute of Technology',
 'headline': 'Data Science | Machine Learning',
 'summary': 'no info',
 'country_full_name': 'United States of America',
 'city': 'Washington DC-Baltimore Area',
 'experiences': [{'starts_at': {'day': 1, 'month': 8, 'year': 2022},
   'ends_at': None,
   'company': 'Georgia Institute of Technology',
   'title': 'Graduate Teaching Assistant',
   'description': 'GTA for the course CS 7641-Machine Learning',
   'location': 'Atlanta, Georgia, United States'},
  {'starts_at': {'day': 1, 'month': 5, 'year': 2022},
   'ends_at': {'day': 31, 'month': 8, 'year': 2022},
   'company': 'GE',
   'title': 'Intern',
   'description': 'Summer Practicum',
   'location': 'Virtual'},
  {'starts_at': {'day': 1, 'month': 7, 'year': 2018},
   'ends_at': {'day': 30, 'month': 9, 'year': 2018},
   'company': 'NCSR "DEMOKRITOS"\u200b',


In [29]:
dict_answer

{'summary': 'Venetis Pallikaras is a Graduate Teaching Assistant at Georgia Institute of Technology with a focus on Data Science and Machine Learning. He has experience working as an intern at GE and NCSR DEMOKRITOS in Greece.',
 'facts': ['Venetis has completed multiple courses and certifications in Data Science, Machine Learning, and Programming.',
  'He is fluent in both English and Greek.',
  'Venetis has experience working with big data technologies such as PySpark and Apache Spark.'],
 'topics_of_interest': ['Data Science',
  'Machine Learning',
  'Big Data Technologies'],
 'ice_breakers': ['What inspired you to pursue a career in Data Science and Machine Learning?',
  'I noticed you have experience working with PySpark and Apache Spark. What was your experience like working with these technologies?']}